In [1]:
#import voor alle benodigde libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns
import plotly.express as px
from sodapy import Socrata

Importing Gekentekende Voertuigen

The source of our API for the dataframe 'voertuigen_df' has their own visualizations.
From these visualizations we can determine that the following columns do not have more than 50% NaN values:

In [41]:
# #Inladen API voertuigen
# #Van de site:
# #https://opendata.rdw.nl/Voertuigen/Open-Data-RDW-Gekentekende_voertuigen/m9d7-ebf2
# client_1 = Socrata("opendata.rdw.nl", None)

# voertuigen = client_1.get("m9d7-ebf2", limit=15200000, where='laadvermogen>0',
#                           select='kenteken, voertuigsoort, merk, handelsbenaming, inrichting, eerste_kleur, aantal_cilinders, massa_ledig_voertuig, wam_verzekerd, aantal_deuren, aantal_wielen, europese_voertuigcategorie, vermogen_massarijklaar, datum_tenaamstelling_dt, datum_eerste_toelating_dt, datum_eerste_tenaamstelling_in_nederland_dt'
#                           )

# voertuigen_df = pd.DataFrame.from_records(voertuigen)

# voertuigen_df.shape

Importing Brandstof Types Voertuigen

The source of our API for the dataframe 'brandstof_df' has their own visualizations.
From these visualizations we can determine that the following columns do not have more than 50% NaN values:

In [26]:
# #Inladen API brandstof types
# #Van de site:
# #https://opendata.rdw.nl/Voertuigen/Open-Data-RDW-Gekentekende_voertuigen_brandstof/8ys7-d773
# client_2 = Socrata("opendata.rdw.nl", None)

# brandstof = client_2.get("8ys7-d773", limit=1300000, where="brandstof_omschrijving='Elektriciteit'",
#                          select='kenteken, brandstof_volgnummer, brandstof_omschrijving, emissiecode_omschrijving, milieuklasse_eg_goedkeuring_licht, klasse_hybride_elektrisch_voertuig, uitlaatemissieniveau'
#                          )

# brandstof_df = pd.DataFrame.from_records(brandstof)

# brandstof_df.shape

Merging voertuigen_df & brandstof_df so we have a dataframe that consists solely of electric vehicles with all their data.

In [52]:
# elektrisch_df = voertuigen_df.merge(brandstof_df, how='inner', on='kenteken')

In [59]:
# elektrisch_df.shape

(19610, 22)

In [54]:
# elektrisch_df.to_csv(r'C:\School\Y3\Semester 2 - Minor\Block 1\Case3\elektrischV2.csv')

Importing an API that provides data on charging docks for cars in the Netherlands

In [101]:
#Inladen API chargemap
response = requests.get("https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&maxresults=1000&compact=true&verbose=false&key=6686970f-0aa6-4cb5-ae81-cf8f6389a816")

#Omzetten naar dictionary
responsejson  = response.json()

#Dataframe bevat kolom die een list zijn. 
#Met json_normalize zet je de eerste kolom om naar losse kolommen
charge_map = pd.json_normalize(responsejson)

charge_map_df = pd.DataFrame.from_records(charge_map)

In [102]:
#Dropping unnecessary columns:
charge_map_df = charge_map_df.drop(columns=['IsRecentlyVerified', 'UUID', 'DataProviderID', 'AddressInfo.CountryID', 
                            'AddressInfo.DistanceUnit', 'AddressInfo.RelatedURL', 'AddressInfo.ContactTelephone1', 
                            'AddressInfo.ContactEmail', 'AddressInfo.ContactTelephone2', 'OperatorsReference', 
                            'DataProvidersReference', 'GeneralComments', 'AddressInfo.AddressLine2',
                            'AddressInfo.AccessComments'
                            ])

In [113]:
#Renaming column names:
charge_map_df.rename(columns={'AddressInfo.Latitude':'LAT', 'AddressInfo.Longitude':'LON', 
                              'AddressInfo.Postcode':'Postcode', 'AddressInfo.ID':'Address_ID', 
                              'AddressInfo.Title':'Address_Title', 'AddressInfo.AddressLine1':'Address_Line1',
                              'AddressInfo.StateOrProvince':'Address_StateOrProvince', 
                              'AddressInfo.Town':'Address_Town'},
                              inplace=True)

In [57]:
laadpaal_df = pd.read_csv('laadpaaldata.csv')
elektrisch_df = pd.read_csv('elektrischV2.csv', index_col=0)

In [58]:
laadpaal_df.head()

,Started,Ended,TotalEnergy,ConnectedTime,ChargeTime,MaxPower
0,2018-01-01 08:45:26,2018-01-01 09:16:45,1830,0.5219,0.5219,3524
1,2018-01-01 11:26:40,2018-01-01 14:36:13,28641,3.1592,2.3056,13680
2,2018-01-01 12:39:28,2018-01-01 20:58:10,65534,8.3117,8.3117,11200
3,2018-01-01 14:26:58,2018-01-02 06:36:55,10080,16.1658,4.3006,3360
4,2018-01-01 14:50:05,2018-01-01 17:21:50,7180,2.5292,2.0656,3758


NOTE TO SELF: NA DE MERGE DOWNLOADEN ALS CSV, DAN HOEFT JE NIET STEEDS 16 MILJOEN TE RUNNEN